In [6]:
import numpy as np
from scipy.optimize import minimize
import tkinter as tk
from tkinter import messagebox
import matplotlib.pyplot as plt

# Define the objective function with checks to avoid division by zero
def objective(x):
    t, h, l, theta = x
    epsilon = 1e-8  # Small value to prevent division by zero
    numerator = 70000 * (t / l) ** 3 * np.cos(theta)
    denominator = ((h / l) + np.sin(theta)) * (np.sin(theta) ** 2 + epsilon)
    if denominator == 0:
        return np.inf  # Avoid division by zero
    return -numerator / denominator / 1e6  

# Define the constraints
def constraint1(x):
    t, h, l, theta = x
    return 0.033333 * h - t  

def constraint2(x):
    t, h, l, theta = x
    return 0.1 * l - t  

def constraint3(x):
    t, h, l, theta = x
    epsilon = 1e-8  # Small value to prevent division by zero
    numerator = 2.7e-09 * ((2 + (h / l)) * t) / l
    denominator = (2 * np.cos(theta)) * ((h / l) + np.sin(theta) + epsilon)
    if denominator == 0:
        return 100  # Avoid division by zero, ensure infeasibility
    return 100 - (numerator / denominator)  # Constraint to be <= 100

# Initial guess
x0 = [0.7, 25.0, 25.0, 0.9]  # Adjusted to be within feasible region

# Bounds for variables
bounds = [(1, 50), (1, 50), (1, 10), (0.01, 1.6)]


constraints = [
    {'type': 'ineq', 'fun': constraint1},
    {'type': 'ineq', 'fun': constraint2},
    {'type': 'ineq', 'fun': constraint3}
]


In [7]:
def run_optimization():
    try:
        # Retrieve bounds and initial guess from the GUI
        t_lower = float(t_lower_entry.get())
        t_upper = float(t_upper_entry.get())
        h_lower = float(h_lower_entry.get())
        h_upper = float(h_upper_entry.get())
        l_lower = float(l_lower_entry.get())
        l_upper = float(l_upper_entry.get())
        theta_lower = float(theta_lower_entry.get())
        theta_upper = float(theta_upper_entry.get())

        initial_t = float(initial_t_entry.get())
        initial_h = float(initial_h_entry.get())
        initial_l = float(initial_l_entry.get())
        initial_theta = float(initial_theta_entry.get())

        # Set bounds for variables
        bounds = [(t_lower, t_upper), (h_lower, h_upper), (l_lower, l_upper), (theta_lower, theta_upper)]

        # Initial guess
        x0 = [initial_t, initial_h, initial_l, initial_theta]

        # Validate initial guess
        initial_constraints = [constraint1(x0), constraint2(x0), constraint3(x0)]
        print(f"Initial guess: {x0}")
        print(f"Initial constraints values: {initial_constraints}")

        if any(c < 0 for c in initial_constraints):
            raise ValueError("Initial guess does not satisfy the constraints")

        # Solve the optimization problem using SLSQP
        solution_slsqp = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=constraints, options={'disp': True})

        # Solve the optimization problem using COBYLA
        solution_cobyla = minimize(objective, x0, method='COBYLA', bounds=bounds, constraints=constraints, options={'disp': True})

        # Compare and display results
        if solution_slsqp.success:
            print('SLSQP Solution:')
            print(solution_slsqp.x)
            print('SLSQP Objective value:')
            print(-solution_slsqp.fun)
        else:
            print('SLSQP Optimization failed:')
            print(solution_slsqp.message)

        if solution_cobyla.success:
            print('COBYLA Solution:')
            print(solution_cobyla.x)
            print('COBYLA Objective value:')
            print(-solution_cobyla.fun)
        else:
            print('COBYLA Optimization failed:')
            print(solution_cobyla.message)

        if not solution_slsqp.success and not solution_cobyla.success:
            raise ValueError("Both optimization methods failed")

    except Exception as e:
        messagebox.showerror("Error", str(e))


In [8]:
# Set up the GUI
root = tk.Tk()
root.title("Optimization")

# Define and place GUI elements
tk.Label(root, text="t lower:").grid(row=0, column=0)
t_lower_entry = tk.Entry(root)
t_lower_entry.grid(row=0, column=1)
t_lower_entry.insert(0, "0.1")

tk.Label(root, text="t upper:").grid(row=1, column=0)
t_upper_entry = tk.Entry(root)
t_upper_entry.grid(row=1, column=1)
t_upper_entry.insert(0, "10")

tk.Label(root, text="h lower:").grid(row=2, column=0)
h_lower_entry = tk.Entry(root)
h_lower_entry.grid(row=2, column=1)
h_lower_entry.insert(0, "1")

tk.Label(root, text="h upper:").grid(row=3, column=0)
h_upper_entry = tk.Entry(root)
h_upper_entry.grid(row=3, column=1)
h_upper_entry.insert(0, "100")

tk.Label(root, text="l lower:").grid(row=4, column=0)
l_lower_entry = tk.Entry(root)
l_lower_entry.grid(row=4, column=1)
l_lower_entry.insert(0, "1")

tk.Label(root, text="l upper:").grid(row=5, column=0)
l_upper_entry = tk.Entry(root)
l_upper_entry.grid(row=5, column=1)
l_upper_entry.insert(0, "100")

tk.Label(root, text="theta lower:").grid(row=6, column=0)
theta_lower_entry = tk.Entry(root)
theta_lower_entry.grid(row=6, column=1)
theta_lower_entry.insert(0, "0")

tk.Label(root, text="theta upper:").grid(row=7, column=0)
theta_upper_entry = tk.Entry(root)
theta_upper_entry.grid(row=7, column=1)
theta_upper_entry.insert(0, str(np.pi / 2))

tk.Label(root, text="Initial t:").grid(row=8, column=0)
initial_t_entry = tk.Entry(root)
initial_t_entry.grid(row=8, column=1)
initial_t_entry.insert(0, "0.7")

tk.Label(root, text="Initial h:").grid(row=9, column=0)
initial_h_entry = tk.Entry(root)
initial_h_entry.grid(row=9, column=1)
initial_h_entry.insert(0, "25")

tk.Label(root, text="Initial l:").grid(row=10, column=0)
initial_l_entry = tk.Entry(root)
initial_l_entry.grid(row=10, column=1)
initial_l_entry.insert(0, "25")

tk.Label(root, text="Initial theta:").grid(row=11, column=0)
initial_theta_entry = tk.Entry(root)
initial_theta_entry.grid(row=11, column=1)
initial_theta_entry.insert(0, "0.9")

tk.Button(root, text="Run Optimization", command=run_optimization).grid(row=12, column=0, columnspan=2)

root.mainloop()


Initial guess: [0.7, 25.0, 25.0, 0.9]
Initial constraints values: [0.13332500000000014, 1.8, np.float64(99.9999999998977)]
Optimization terminated successfully    (Exit mode 0)
            Current function value: -8.729171822067008e-07
            Iterations: 1
            Function evaluations: 5
            Gradient evaluations: 1
SLSQP Solution:
[ 0.7 25.  25.   0.9]
SLSQP Objective value:
8.729171822067008e-07
COBYLA Solution:
[ 9.44959005e-01  2.83490536e+01  9.44959005e+00 -8.47032947e-26]
COBYLA Objective value:
2333.3100000000013
